In [ ]:
import { StructuredOutputParser } from "@langchain/core/output_parsers";
import { PromptTemplate } from "@langchain/core/prompts";
import { ChatOpenAI } from "@langchain/openai";

const llm = new ChatOpenAI({
  modelName: process.env.MODEL_NAME,
  configuration: {
    baseURL: process.env.BASE_URL,
    apiKey: process.env.OPEN_API_KEY,
  },
});

const parser = StructuredOutputParser.fromNamesAndDescriptions({
  answer: "用户问题的答案，格式是以英文逗号隔开的中文字符串",
  evidence: "你回答用户问题所依据的答案",
  confidence: "问题答案的可信度评分，格式是百分数",
});
// parser.getFormatInstructions();

const prompt = PromptTemplate.fromTemplate(`
    列出3个 {country} 的著名的互联网公司.
    {instructions}
`);
const chain = prompt.pipe(llm).pipe(parser);
await chain.invoke({
  instructions: parser.getFormatInstructions(),
  country: "中国",
});


In [ ]:
import { CommaSeparatedListOutputParser } from "@langchain/core/output_parsers";
import { PromptTemplate } from "@langchain/core/prompts";
import { ChatOpenAI } from "@langchain/openai";

const llm = new ChatOpenAI({
  modelName: process.env.MODEL_NAME,
  configuration: {
    baseURL: process.env.BASE_URL,
    apiKey: process.env.OPEN_API_KEY,
  },
});

const parser = new CommaSeparatedListOutputParser();
// parser.getFormatInstructions();

const prompt = PromptTemplate.fromTemplate(`
    列出 {country} 排名前20的互联网公司.
    {instructions}
`);
const chain = prompt.pipe(llm).pipe(parser);
await chain.invoke({
  instructions: parser.getFormatInstructions(),
  country: "中国",
});


In [ ]:
import {
  StructuredOutputParser,
  OutputFixingParser,
} from "langchain/output_parsers";
import { PromptTemplate } from "@langchain/core/prompts";
import { ChatOpenAI } from "@langchain/openai";
import { z } from "zod";

const llm = new ChatOpenAI({
  modelName: process.env.MODEL_NAME,
  configuration: {
    baseURL: process.env.BASE_URL,
    apiKey: process.env.OPEN_API_KEY,
  },
});

const schema = z.object({
  answer: z.string().describe("用户问题的答案，以英文逗号隔开的3个中文字符串"),
  confidence: z
    .number()
    .min(0)
    .max(100)
    .describe("问题答案的可信度评分，满分 100"),
});

const parser = StructuredOutputParser.fromZodSchema(schema);

// const prompt = PromptTemplate.fromTemplate(`
//     列出3个 {country} 的著名的互联网公司.
//     {instructions}
// `);
// const chain = prompt.pipe(llm).pipe(parser);
// await chain.invoke({
//   instructions: parser.getFormatInstructions(),
//   country: "中国",
// });

const wrongOutput = {
  abc: ["阿里巴巴", "腾讯", "百度", "字节"],
  def: "90%",
};
const fixParser = OutputFixingParser.fromLLM(llm, parser);
await fixParser.parse(JSON.stringify(wrongOutput));


In [ ]:
// import { TextLoader } from "langchain/document_loaders/fs/text";
import { JSONLoader } from "langchain/document_loaders/fs/json";

// const loader = new TextLoader('documents/data.txt')
// const docs = await loader.load()
// console.log(docs)

const jsonLoader = new JSONLoader("documents/data.json", "/title");
// const jsonLoader = new JSONLinesLoader('data.jsonl', '/html')
const json = await jsonLoader.load();
console.log(json);


In [ ]:
import { PDFLoader } from "@langchain/community/document_loaders/fs/pdf";
import "pdf-parse";

const loader = new PDFLoader("documents/data.pdf");
const docs = await loader.load();
console.log(docs);


In [ ]:
import { DirectoryLoader } from "langchain/document_loaders/fs/directory";
import { TextLoader } from "langchain/document_loaders/fs/text";
import { PDFLoader } from "@langchain/community/document_loaders/fs/pdf";
import "pdf-parse";

const loader = new DirectoryLoader("./documents", {
  ".txt": (path) => new TextLoader(path),
  ".pdf": (path) => new PDFLoader(path, { splitPages: false }),
});
const docs = await loader.load();

console.log(docs);


In [ ]:
import { DocxLoader } from "@langchain/community/document_loaders/fs/docx";
import "mammoth";

const loader = new DocxLoader("documents/data.docx");

const docs = await loader.load();

console.log(docs);


In [ ]:
import { PPTXLoader } from "@langchain/community/document_loaders/fs/pptx";
import "officeparser";

const loader = new PPTXLoader("documents/data.pptx");

const docs = await loader.load();

console.log(docs);


In [ ]:
import { GithubRepoLoader } from "@langchain/community/document_loaders/web/github";
import "ignore";

const loader = new GithubRepoLoader(
  "https://github.com/Issho-lin/webrtc-demo",
  {
    branch: "main",
    recursive: false,
    unknown: "warn",
    ignorePaths: ["*.md", "*.json"],
  }
);
const docs = await loader.load();

console.log(docs);


In [ ]:
import { CheerioWebBaseLoader } from "@langchain/community/document_loaders/web/cheerio";
import "cheerio";

const loader = new CheerioWebBaseLoader(
  "https://kaiyi.cool/blog/github-copilot",
  {
    selector: "h3",
  }
);

const docs = await loader.load();

console.log(docs[0].pageContent);


In [10]:
import { SerpAPILoader } from "@langchain/community/document_loaders/web/serpapi"

const apiKey = '185032b32dc536d633129d221f7f7be48629f0e4569138241d817449166e21ca'
const question = "什么 github copliot"
const loader = new SerpAPILoader({ q: question, apiKey });
const docs = await loader.load();

console.log(docs)

[
  Document {
    pageContent: '{"type":"organic_result","title":"有关使用GitHub Copilot 的最佳做法","link":"https://docs.github.com/zh/copilot/using-github-copilot/best-practices-for-using-github-copilot#:~:text=GitHub%20Copilot%20%E6%98%AF%E4%B8%80%E6%AC%BE,%E9%97%AE%E9%A2%98%E8%A7%A3%E5%86%B3%E5%92%8C%E5%8D%8F%E4%BD%9C%E4%B8%8A%E3%80%82","displayed_link":"https://docs.github.com › copilot › using-github-copilot","snippet":"GitHub Copilot 是一款AI 编码助手，可帮助你更快、更省力地编写代码，从而将更多精力集中在问题解决和协作上。","snippet_highlighted_words":["是一款AI 编码助手"],"favicon":"https://serpapi.com/searches/680dcb99bcbfc220697e3625/images/ddf35b21095b25aa0e374bb30fccb487d5eb98f3a8c7a7fc.png","source":"GitHub Docs"}',
    metadata: { source: "SerpAPI", responseType: "answer_box" },
    id: undefined
  },
  Document {
    pageContent: '{"title":"GitHub Copilot","type":"Software","entity_type":"webanswers","kgmid":"/g/11q83qbj3d","knowledge_graph_search_link":"https://www.google.com/search?kgmid=/g/11q83qbj3d&hl=en-US&q=GitHub+Copilot